# CKME136 - Capstone Project - Toronto Real Estate Listings
## Step 2: Feature Engineering - with Pandas

<div class="alert alert-block alert-info">
2A. Imports for feature engineering

In [1]:
import csv
import datetime
import pandas as pd
from ast import literal_eval

<div class="alert alert-block alert-info">
2B. Read in listing other features

In [2]:
# read in individual_listing_other_Features
df_other_features = pd.read_csv('individual_listing_other_features_v3.csv')
# argument to convert string to list on read
# converters={"listing_features": literal_eval})

# read in individual_listing_rooms
df_rooms = pd.read_csv('individual_listing_rooms.csv')

<div class="alert alert-block alert-info">
2C. Use Pandas to transform listing other features data for predictions

In [3]:
# extract mls id
df_other_features['listing_id'] = df_other_features['listing_url'].str.extract(r'[A-Z](\d{7})', expand=True)

# convert dollars to int
def convert_dollar_string_to_int(column):
    df_other_features[column+'_int'] = df_other_features[column].replace('[\$,]', '', regex=True).astype(int)

convert_dollar_string_to_int('listing_price')
convert_dollar_string_to_int('comparable_sold_price_1')
convert_dollar_string_to_int('comparable_list_price_1')
convert_dollar_string_to_int('comparable_sold_price_2')
convert_dollar_string_to_int('comparable_list_price_2')
convert_dollar_string_to_int('comparable_sold_price_3')
convert_dollar_string_to_int('comparable_list_price_3')
convert_dollar_string_to_int('comparable_sold_price_4')
convert_dollar_string_to_int('comparable_list_price_4')
convert_dollar_string_to_int('comparable_sold_price_5')
convert_dollar_string_to_int('comparable_list_price_5')

# extract postal code via one-hot encoding
# df_other_features['postal_code'] = df_other_features['address'].str.split(',', expand = True)[2].str.split(expand = True)[1]
# df_other_features['postal_code_fsa'] = df_other_features['address'].str.split(',', expand = True)[2].str.split(expand = True)[1].astype(str).str[:3]
# df_other_features['postal_code_ldu'] = df_other_features['address'].str.split(',', expand = True)[2].str.split(expand = True)[1].astype(str).str[3:]
df_series_fsa = df_other_features['address'].str.split(',', expand = True)[2].str.split(expand = True)[1].astype(str).str[:3]

df_postal_code_fsa = pd.get_dummies(df_series_fsa)

# extract community
# df_other_features['community'] = df_other_features['address'].str.split(',', expand = True)[1]
df_series_community = df_other_features['address'].str.split(',', expand = True)[1]
df_community = pd.get_dummies(df_series_community)

# building type
# df_other_features['building_type'] = df_other_features['dwelling_type'].str.split(',', expand = True)[0]
df_series_building_type = df_other_features['dwelling_type'].str.split(',', expand = True)[0]
df_building_type = pd.get_dummies(df_series_building_type)

# number of room features
df_other_features['main_rooms'] = df_other_features['dwelling_type'].str.split(',', expand = True)[1].str.split(pat = '(\d+)', expand = True)[1]
df_other_features['additional_rooms'] = df_other_features['dwelling_type'].str.split(',', expand = True)[1].str.split(pat = '(\d+)', expand = True)[3]

# change to numbers and fill
df_other_features['main_rooms'] = pd.to_numeric(df_other_features.main_rooms).fillna(0).astype(int)
df_other_features['additional_rooms'] = pd.to_numeric(df_other_features.additional_rooms).fillna(0).astype(int)

# generate total rooms
df_other_features['total_rooms'] = df_other_features['main_rooms'] + df_other_features['additional_rooms']

# of parking spaces
df_other_features['parking_spaces'] = df_other_features['dwelling_type'].str.split(',', expand = True)[2].str.split(pat = 'Parking', expand = True)[0].str.split(pat = 'Kitchen|Kitchens', expand = True)[1]
# this coerce may lose us some data as listings with parking space 's1' will be converted to 0
df_other_features['parking_spaces'] = pd.to_numeric(df_other_features.parking_spaces, errors='coerce').fillna(0).astype(int)

# of kitchens
df_other_features['main_kitchen'] = df_other_features['dwelling_type'].str.split(',', expand = True)[2].str.split(pat = 'Parking', expand = True)[0].str.split(pat = 'Kitchen|Kitchens', expand = True)[0].str.split(pat = '+', expand = True)[0]
df_other_features['additional_kitchen'] = df_other_features['dwelling_type'].str.split(',', expand = True)[2].str.split(pat = 'Parking', expand = True)[0].str.split(pat = 'Kitchen|Kitchens', expand = True)[0].str.split(pat = '+', expand = True)[1]

df_other_features['main_kitchen'] = pd.to_numeric(df_other_features.main_kitchen).fillna(0).astype(int)
df_other_features['additional_kitchen'] = pd.to_numeric(df_other_features.additional_kitchen).fillna(0).astype(int)
df_other_features['total_kitchens'] = df_other_features['main_kitchen'] + df_other_features['additional_kitchen']

# convert string dates to datetime
def convert_string_to_date(column):
#     df_other_features[column+'_datetime'] = pd.to_datetime(df_other_features[column])
    a = pd.to_datetime(df_other_features[column], errors = 'coerce')
    df_other_features[column+'_datetime'] = a.apply(lambda x: datetime.datetime.now() - x).astype(int)
    
convert_string_to_date('comparable_sold_date_1')
convert_string_to_date('comparable_sold_date_2')
convert_string_to_date('comparable_sold_date_3')
convert_string_to_date('comparable_sold_date_4')
convert_string_to_date('comparable_sold_date_5')

# extras
df_other_features['extras_fridge'] = df_other_features['listing_extras'].str.lower().str.contains('fridge').fillna(0).astype(int)
df_other_features['extras_stove'] = df_other_features['listing_extras'].str.lower().str.contains('stove').fillna(0).astype(int)
df_other_features['extras_stainless_steel'] = df_other_features['listing_extras'].str.lower().str.contains('stainless steel').fillna(0).astype(int)
df_other_features['extras_pool'] = df_other_features['listing_extras'].str.lower().str.contains('pool').fillna(0).astype(int)
df_other_features['extras_gym'] = df_other_features['listing_extras'].str.lower().str.contains('gym').fillna(0).astype(int)
df_other_features['extras_storage'] = df_other_features['listing_extras'].str.lower().str.contains('storage').fillna(0).astype(int)
df_other_features['extras_locker'] = df_other_features['listing_extras'].str.lower().str.contains('locker').fillna(0).astype(int)
df_other_features['extras_gas'] = df_other_features['listing_extras'].str.lower().str.contains('gas').fillna(0).astype(int)
df_other_features['extras_dishwasher'] = df_other_features['listing_extras'].str.lower().str.contains('dishwasher').fillna(0).astype(int)

# features
df_other_features['features_view'] = df_other_features['listing_features'].str.lower().str.contains('view').fillna(0).astype(int)
df_other_features['features_park'] = df_other_features['listing_features'].str.lower().str.contains('park').fillna(0).astype(int)
df_other_features['features_public_transit'] = df_other_features['listing_features'].str.lower().str.contains('public transit').fillna(0).astype(int)
df_other_features['features_basementbrick'] = df_other_features['listing_features'].str.lower().str.contains('basementbrick').fillna(0).astype(int)
df_other_features['features_concrete_exterior'] = df_other_features['listing_features'].str.lower().str.contains('concrete exterior').fillna(0).astype(int)
df_other_features['features_hospital'] = df_other_features['listing_features'].str.lower().str.contains('hospital').fillna(0).astype(int)
df_other_features['features_library'] = df_other_features['listing_features'].str.lower().str.contains('library').fillna(0).astype(int)

# description length
df_other_features['listing_description_length'] = df_other_features['listing_description'].str.len().fillna(0).astype(int)

In [4]:
# create clean dataset for other features
df_other_features_listing_url = df_other_features.iloc[:,0:1]
df_other_features_dataset = df_other_features.iloc[:,23:]
df_other_features_frames = [df_other_features_listing_url, df_other_features_dataset, df_postal_code_fsa, df_community, df_building_type]
df_other_features_dataset = pd.concat(df_other_features_frames, axis=1)

<div class="alert alert-block alert-info">
2D. Use Pandas to transform rooms features data for predictions

In [5]:
# boolean_rooms
df_rooms_bool = df_rooms.notna()
#select specific rows
df_rooms_bool = df_rooms_bool.iloc[:,1:]
df_rooms_listing_url = df_rooms.iloc[:,0:1]
#join dataframes
frames = [df_rooms_listing_url, df_rooms_bool]
df_rooms_dataset = pd.concat(frames, axis=1)

<div class="alert alert-block alert-info">
2E. Join datasets together and write to CSV

In [6]:
# Join rooms and other features datasets together
df_all_features_dataset = df_other_features_dataset.merge(df_rooms_dataset, on = 'listing_url')

In [7]:
# Check data types of all features
pd.set_option('display.max_rows', 500)
df_all_features_dataset.dtypes

listing_url                             object
listing_id                              object
listing_price_int                        int64
comparable_sold_price_1_int              int64
comparable_list_price_1_int              int64
comparable_sold_price_2_int              int64
comparable_list_price_2_int              int64
comparable_sold_price_3_int              int64
comparable_list_price_3_int              int64
comparable_sold_price_4_int              int64
comparable_list_price_4_int              int64
comparable_sold_price_5_int              int64
comparable_list_price_5_int              int64
main_rooms                               int64
additional_rooms                         int64
total_rooms                              int64
parking_spaces                           int64
main_kitchen                             int64
additional_kitchen                       int64
total_kitchens                           int64
comparable_sold_date_1_datetime          int64
comparable_so

In [8]:
# Write to CSV
df_all_features_dataset.to_csv('cleaned_combined_listing_features_v1.csv')

In [286]:
x = df_all_features_dataset
x.head()
x = df_all_features_dataset.iloc[:,3:]
y = df_all_features_dataset['listing_price_int']

In [288]:
# Create model
lm = linear_model.LinearRegression()
model = lm.fit(x,y)

In [289]:
# Make some predictions
predictions = lm.predict(x)
print(predictions)[0:10]

[ 653818.97310888  454684.17603388  624876.90133691  950296.04379923
  790803.57326784 1087296.49442019  402155.61275966 1430847.26785217
 1598437.97814546  448151.02919796]


In [290]:
# Actual listing prices
y.head(10)

0     899900
1     425000
2     479900
3     949000
4     680000
5     799000
6     349900
7    1878000
8    1999000
9     549000
Name: listing_price_int, dtype: int64

In [291]:
# r squared score for model
lm.score(x,y)

0.6719553386883852

In [293]:
# Create training and test dataset
# train, test = train_test_split(df_all_features_dataset, test_size=0.3)


# Labels are the values we want to predict
labels = np.array(df_all_features_dataset['listing_price_int'])
# Remove the labels from the features
# axis 1 refers to the columns
features = df_all_features_dataset.drop('listing_price_int', axis = 1).drop('listing_url', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [294]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [295]:
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [296]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
# errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
# print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Mean Absolute Error: 3.83 degrees.

In [297]:
predictions[:5]

array([328561.421, 683096.068, 487202.397, 427464.975, 491474.355])

In [300]:
test_labels[:5]

array([269900, 725000, 519900, 399950, 488000])

In [302]:
# !pip install geopy

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="specify_your_app_name_here")
# location = geolocator.geocode("88 Park Lawn Rd 2717, Mimico, Toronto M8Y0B5")
# print(location.address)
# print((location.latitude, location.longitude))

In [303]:
# find the term Lot Size
# soup.find("div", {"class": "lpc15"}).findNext("div", {"class": "lpc15"}).findNext("div", {"class": "lpc15"}).find_previous_sibling()
# soup.find('div', attrs={'style': 'color: #3a5fac; padding-top: 5px; font-size: 18px; '})